# RMQ Segment Trees – Compact Revision Guide

## Types
- **Min Segment Tree** → stores minimums, natural for ≤ or < k queries.
- **Max Segment Tree** → stores maximums, natural for ≥ or> k queries.
- **Sum Segment Tree** → stores sums, used for range totals/counts.
- **Generalized Segment Tree** → any associative function (min, max, sum, gcd, xor).

## Supported Queries
### Aggregate (classic 3‑case recursion)
- Range Minimum Query (RMQ)
- Range Maximum Query
- Range Sum Query
- Range GCD / LCM / XOR

### Guided Search (using min/max to prune)
- Earliest index ≤ k (min tree)
- Latest index ≤ k (min tree)
- Earliest index ≥ k (max tree)
- Latest index ≥ k (max tree)
- First index < threshold (min tree)
- First index > threshold (max tree)

### Updates
- **Point update** → change one element, recompute path.
- **Range update (lazy propagation)** → add/assign to all elements in [l,r].

## Complexity
- Build: **O(n)**
- Query: **O(log n)**
- Point update: **O(log n)**
- Lazy range update/query: **O(log n)**

## When to Use
- **Min tree** → positional queries for ≤ k.
- **Max tree** → positional queries for ≥ k.
- **Sum tree** → counts, prefix sums, totals.
- **Merge sort tree** → supports “count ≤ k” queries but heavier:  
  - Space: \(O(n \log n)\)  
  - Query: \(O(\log^2 n)\)  
  - Slower and more memory‑intensive than RMQ trees.
- **Fenwick tree (BIT)** → lightweight for prefix/range counts.
- **Wavelet tree** → most general (rank/select, kth element), but complex to implement.

---

📌 **Takeaway:** RMQ trees are a **general framework**.  
Swap the aggregate (min, max, sum, gcd, xor) to change query type.  
Use guided descent for positional queries, lazy propagation for range updates.  
Remember: merge sort trees handle count queries but are heavier and slower.


In [ ]:
# for standard sum in a range queries
class Node:
    def __init__(self,s,e,val:int=0,left=None,right=None):
        self.s = s
        self.e = e
        self.val = val
        self.left = left
        self.right = right
class SegmentTree():
    def __init__(self,arr):
        self.arr = arr
        self.root = self.create(0,len(self.arr)-1)
        
    
    def create(self,s,e):
        
        if s == e:
            return Node(s,e,self.arr[s])
        
        node = Node(s,e)
        m = s + (e-s)//2
        node.left = self.create(s,m)
        node.right = self.create(m+1,e)
        a = node.left.val if node.left else 0
        b = node.right.val if node.right else 0
        node.val = a + b
        return node

    def sum(self,s,e,node=None):
        if node is None:
            node = self.root
            
        s1,e1 = node.s,node.e
        if s1>e1:
            return 0
        
        if s1>=s and e1<=e:
            return node.val
        elif e1<s or s1>e:
            return 0
        else:
            return self.sum(s,e,node.left) + self.sum(s,e,node.right)

arr = [1,2,3,4,5,6,7,8,9]

tree = SegmentTree(arr)

print(tree.sum(1,5))
    
        
        

In [ ]:
# for count of elements less than or equal to k in range queries
class Node:
    def __init__(self,s,e,val:list=[],left=None,right=None):
        self.s = s
        self.e = e
        self.val = val
        self.left = left
        self.right = right
class MergeSortSegmentTree():
    def __init__(self,arr):
        self.arr = arr
        self.root = self.create(0,len(self.arr)-1)
        
    
    def create(self,s,e):
        
        if s == e:
            return Node(s,e,[self.arr[s]])
        
        node = Node(s,e)
        m = s + (e-s)//2
        node.left = self.create(s,m)
        node.right = self.create(m+1,e)
        a = node.left.val if node.left else []
        b = node.right.val if node.right else []
        node.val = self.merge(a,b)
        return node

    def query(self,s,e,k,node=None):
        if node is None:
            node = self.root
            
        s1,e1 = node.s,node.e
        if s1>e1:
            return 0
        
        if s1>=s and e1<=e:
            
            return self.binarySearch(node.val,k)
        elif e1<s or s1>e:
            return 0
        else:
            return self.query(s,e,k,node.left) + self.query(s,e,k,node.right)
    
    def merge(self,a,b):
        n1 = len(a)
        n2 = len(b)
        i,j = 0,0
        ans = []
        while i<n1 and j<n2:
            if a[i]<b[j]:
                ans.append(a[i])
                i += 1
            else:
                ans.append(b[j])
                j += 1
        while i<n1:
            ans.append(a[i])
            i += 1
        while j<n2:
            ans.append(b[j])
            j += 1
        return ans

    def binarySearch(self,arr,k):
        s,e = 0,len(arr)-1
        ans = -1
        while(s<=e):
            m = s + (e-s)//2
            if arr[m]<=k:
                ans = m
                s = m+1
            else:
                e = m-1
        if ans == -1:
            return 0
        return ans + 1

arr = [8,9,1,5,6,7,2,3,4,0]


tree = MergeSortSegmentTree(arr)

print(tree.query(0,4,3)) 
print(tree.query(0,9,3))  
print(tree.query(5,9,6))  

    
        
        

1
4
4


In [6]:
# for maximum in a range queries
class Node:
    def __init__(self,s,e,val:float=float("-inf"),left=None,right=None):
        self.s = s
        self.e = e
        self.val = val
        self.left = left
        self.right = right
class RangeMaximumTree():
    def __init__(self,arr):
        self.arr = arr
        self.root = self.create(0,len(self.arr)-1)
        
    
    def create(self,s,e):
        
        if s == e:
            return Node(s,e,self.arr[s])
        
        node = Node(s,e)
        m = s + (e-s)//2
        node.left = self.create(s,m)
        node.right = self.create(m+1,e)
        a = node.left.val if node.left else float("-inf")
        b = node.right.val if node.right else float("-inf")
        node.val = max(a,b)
        return node

    def max(self,s,e,node=None):
        if node is None:
            node = self.root
            
        s1,e1 = node.s,node.e
        if s1>e1:
            return float("-inf")
        
        if s1>=s and e1<=e:
            return node.val
        elif e1<s or s1>e:
            return float("-inf")
        else:
            return max(self.max(s,e,node.left),self.max(s,e,node.right))

arr = [8,9,1,5,6,7,2,3,4,0]

tree = RangeMaximumTree(arr)

print(tree.max(1,5))
    
        
        

9


In [ ]:
class Node:
    def __init__(self, s, e, val=float("inf"), left=None, right=None):
        self.s = s
        self.e = e
        self.val = val
        self.left = left
        self.right = right

class RangeMinimumTree:
    def __init__(self, arr):
        self.arr = arr
        self.root = self.create(0, len(self.arr)-1)
        
    def create(self, s, e):
        if s == e:
            return Node(s, e, self.arr[s])
        
        m = s + (e-s)//2
        left = self.create(s, m)
        right = self.create(m+1, e)
        node = Node(s, e, min(left.val, right.val), left, right)
        return node

    def earliest_leq(self, s, e, k, node=None):
        if node is None:
            node = self.root
        
        # completely outside
        if node.e < s or node.s > e:
            return -1
        
        # no candidate in this segment. If min>k all are greter than k so reject
        if node.val > k:
            return -1
        
        # leaf node
        if node.s == node.e:
            return node.s
        
        # search left first
        left_ans = self.earliest_leq(s, e, k, node.left)
        if left_ans != -1:
            return left_ans
        return self.earliest_leq(s, e, k, node.right)

# Example
arr = [8,9,1,5,6,7,2,3,4,0]
tree = RangeMinimumTree(arr)

print(tree.earliest_leq(1,5,5))  # earliest index in [1,5] with value <= 5


2
